
# Nitsche based enforcement of Dirichlet boundary conditions for the Hodge Laplacian
## Author : Camilo Tello Fachin
### First implementation for Masters Thesis
This is a first Juptyter Notebook with an implementation for the Hodge Laplacian in 2D for 1-forms.

Initial Problem, The Hodge-Laplacian in the continuous setting.

Let $\Omega \subset \mathbb{R}^d, d= 2,3$ be a bounded Lipschitz domain with $\Gamma$ as its boundary. Formally, We seek a $k$-form $\omega$ such that

$$
\begin{align}
    (\delta \text{d} + \text{d} \delta)\omega &= f,  \quad \quad \text{in } \Omega, \tag{1a}\\
    \text{tr}(\omega) &= 0, \quad \quad \text{on } \Gamma,  \tag{1b} \\
    \text{tr} (\star \omega) &= 0, \quad \quad \text{on } \Gamma. \tag{1c}
\end{align}
$$

Classically, problems of such character are solved using a mixed formulation. However, due to the conflicting boundary conditions $\text{tr}(\star \omega) = 0$ and $\text{tr}(\star \text{d}\omega) = 0$ arising from the mixed formulation, a conforming discretizaion is out of the question. Instead here implemented, a Nitsche type method that enforces (1b) via a penalty term.

We seek $\omega_h \in \Lambda^k_h(\Omega)$ and $\sigma_h \in \Lambda^{k-1}_h(\Omega)$ such that

$$
\begin{align}
    \langle \sigma_h , \tau_h \rangle_{L^2(\Omega)} - \langle \omega_h , \text{d} \tau_h \rangle_{L^2(\Omega)} &= 0, \tag{2a} \\[12pt]

    \langle \text{d} \sigma_h , \eta_h \rangle_{L^2(\Omega)} + \langle \text{d} \omega_h , \text{d} \text{d} \eta_h \rangle_{L^2(\Omega)} \tag{2b} \\[7pt]
    
    + \int_{\Gamma} \text{tr}(\star \text{d} \omega_h) \wedge \text{tr}(\eta_h) +  \int_{\Gamma} \text{tr}(\omega_h) \wedge \text{tr}(\star \text{d} \eta_h) \tag{2c} \\[7pt]

    + \frac{\text{C}_{\omega}}{\text{h}} \langle \text{tr} \omega_h , \text{tr} \eta_h \rangle_{L^2(\Gamma)} &= \langle f, \eta_h \rangle_{L^2(\Omega)}. \tag{2d} 


\end{align}
$$

$\forall \eta_h \in \Lambda^k_h(\Omega)$ and $ \forall \tau_h \in \Lambda^{k-1}_h(\Omega)$.

In this notebook, we will see that when $d=2$ and $k=1$,  the differential form spaces $\Lambda^k_h(\Omega)$ and $\Lambda^{k-1}_h(\Omega)$  can be implemented using Sobolev spaces. Specifically, these spaces correspond to the discrete counterparts of the Sobolev spaces $H(\text{curl}, \Omega)$ and $H^1(\Omega)$, respectively, when expressed through their vector proxies.

In [1]:
# Load Things! 234

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
import scipy.sparse as sp
import numpy as np

In [2]:
# Relevant parameters to play around with
order = 1 # mesh order
max_h = 0.1 # mesh size
C_w = 630# penalty term weight

In FEEC, the de Rham complex provides a framework that ensures the exactness of sequences between differential form spaces and their Sobolev space counterparts. Below defined, the differential form spaces and their associated Sobolev spaces, which preserve the structure necessary for the discretization process.

$$

0 \longrightarrow H\Lambda^0(\Omega) \xrightarrow{d} H\Lambda^1(\Omega) \xrightarrow{d} \dots \xrightarrow{d} H\Lambda^n(\Omega) \longrightarrow 0

$$

$$

0 \longrightarrow H^1(\Omega) \xrightarrow{\text{grad}} H(\text{curl}, \Omega) \xrightarrow{\text{curl}} H(\text{div}, \Omega) \xrightarrow{\text{div}} L^2(\Omega) \longrightarrow 0

$$

Here in this first Jupyter Notebook:
- $H^1(\Omega)$ are 0-forms or scalars
- $H(\text{curl}, \Omega)$ are 1-forms or vectorfiels

In NGSolve, it is standard procedure to introduce a product space for mixed formulations, this translates to adding equations (2a) and (2bcd).

In [3]:
mesh = Mesh(unit_square.GenerateMesh(maxh=max_h))
fes_curl = HCurl(mesh, order=order, type1=False)  # For 1-forms, H(curl)
fes_H1 = H1(mesh, order=order)     # For 0-forms, H1 space

fes = fes_curl * fes_H1 # Product space

(omega, sigma), (eta, tau) = fes.TnT()

#help(fes_curl)

Help on HCurl in module ngsolve.comp object:

class HCurl(FESpace)
 |  Keyword arguments can be:
 |  
 |  order: int = 1
 |    order of finite element space
 |  complex: bool = False
 |    Set if FESpace should be complex
 |  dirichlet: regexpr
 |    Regular expression string defining the dirichlet boundary.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet = 'top|right'
 |  dirichlet_bbnd: regexpr
 |    Regular expression string defining the dirichlet bboundary,
 |    i.e. points in 2D and edges in 3D.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet_bbnd = 'top|right'
 |  dirichlet_bbbnd: regexpr
 |    Regular expression string defining the dirichlet bbboundary,
 |    i.e. points in 3D.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet_bbbnd = 'top|right'
 |  definedon: Region or regexpr
 |    FESpace is only defined on specific Region, created with mesh.Materials('regexpr')
 

Defined and added to the bilinear form object in the cell below:

$$

\langle \sigma_h , \tau_h \rangle_{L^2(\Omega)} - \langle \omega_h , \text{d} \tau_h \rangle_{L^2(\Omega)} \tag{2a}

$$

In [4]:
a = BilinearForm(fes)

a += sigma * tau * dx
a += - omega * grad(tau) * dx

$$

+ \, \langle \text{d} \sigma_h , \eta_h \rangle_{L^2(\Omega)} + \langle \text{d} \omega_h , \text{d} \eta_h \rangle_{L^2(\Omega)} \tag{2b}

$$

In [5]:
a +=  grad(sigma) * eta * dx
a +=  curl(omega) * curl(eta) * dx

Lets unpack the first boundary term from (2c):

$$
+ \, \int_{\Gamma} \text{tr}(\star d\omega_h) \wedge \text{tr}(\eta_h) \tag{2c.1}
$$

If we consider the k-forms in 2D to be: 

$$
\omega , \eta \in \Lambda^1(\Omega) \, \Longleftrightarrow \, \omega, \eta \in H(\text{curl}, \Omega)
$$

We can plug in the according vector calculus operations and transform them with help from some identities:

$$
\begin{aligned}
&= \int_{\Gamma} (\text{n} \times \nabla \times \omega_h) \cdot \eta_h \\[10pt]

&= \int_{\Gamma} (\nabla \times \omega_h) \cdot (\eta_h \times \text{n}) \cdot e_z\\[10pt]

&= \int_{\Gamma} \text{rot}(\omega_h) (\eta_h \times \text{n}) \cdot e_z \\[10pt]

&= \int_{\Gamma} (\partial_1 \, \omega_h^2 - \partial_2 \, \omega_h^1) (\eta_h^2 \, \text{n}^1 - \eta_h^1 \, \text{n}^2) 
\end{aligned}
$$

- superscripts in the last line indicate the components of $\omega_h$
- Exterior Derivative of $\omega$ is the curl, transforms 1-form $\omega$ to a 2-form.
- Hodge Star operator $\star$ transforms 2-form $\omega$ to a n-k or here a 0-form.
- The wedge product $\wedge$ of a 0-form (curl of $\omega$) and the trace of a 1-form 
  $\text{tr}(\eta)$ is simply a multiplication of a scalar by a tangential component.



In [6]:
n = specialcf.normal(mesh.dim)
t = specialcf.tangential(mesh.dim)

a += -curl(omega) * eta.Trace()*t * ds(skeleton = True, definedon=mesh.Boundaries(".*"))

And the second boundary term from (2c):

$$
+ \int_{\Gamma} \text{tr} ( \omega_h ) \wedge \text{tr} ( \star d\eta_h ) \tag{2c.2}
$$

Again plugging in stuff and do the according computations that are the same but the other way around:

$$
\begin{aligned}

&=\int_{\Gamma} (\omega_h \times \text{n}) \cdot (\nabla \times \eta_h) \\[10pt]

&= \int_{\Gamma} (\omega_h \times \text{n}) \cdot e_z \cdot \text{rot}(\eta_h) \\[10pt]

&= \int_{\Gamma} (\omega_h^2 \, \text{n}^1 - \omega_h^1 \, \text{n}^2)  (\partial_1 \, \eta_h^2 - \partial_2 \, \eta_h^1)

\end{aligned}
$$



In [7]:
a += omega.Trace()* t * curl(eta) *  ds(skeleton=True, definedon=mesh.Boundaries(".*"))

And the pentalty or stabilization term:

$$
+ \, \frac{C_w}{h} \langle \text{tr}\, \omega, \text{tr}\, \eta \rangle_{L^2(\Gamma)} \tag{2d.1}
$$

In [8]:
h = specialcf.mesh_size #computed on every edge of the boundary integration is way faster than setting a constant
a += (C_w / h) * omega.Trace() * t * eta.Trace() * t * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

For the right hand side $f$ and the last part of (2d), we introduce a manufactured solution. This one us also used to compute $\lVert \omega_h - \omega \rVert_{L^2(\Omega)}$. The manufactured solution $\omega$ is chosen such that it vanishes on the boundary. 

$$
\omega = \begin{pmatrix} \sin(\pi x) \sin(\pi y) \\ \sin(\pi x) \sin(\pi y) \end{pmatrix}
$$

In order to incorporate it to the right hand side linear form, we can simply just caluclate the Hodge Laplacian of it.

$$
\Delta_{\text{H}}\omega = (d \delta \omega + \delta d \omega) = \text{curl}(\text{rot}(\omega)) - \text{grad}(\text{div}(\omega)) = \text{f}
$$

This computed quantity we can now plug into:

$$

\langle f, \eta_h \rangle_{L^2(\Omega)}. \tag{2d.2} 

$$

In [9]:
omega_exact = CF((sin(pi*x)*sin(pi*y), sin(pi*x)*sin(pi*y)))

div_omega = CF(omega_exact[0].Diff(x) + omega_exact[1].Diff(y)) # take this *(-1) and you one has sigma manufactured!
grad_div_omega = CF((div_omega.Diff(x), div_omega.Diff(y)))

rot_omega = CF(omega_exact[1].Diff(x) - omega_exact[0].Diff(y))
curl_rot_omega = CF((rot_omega.Diff(y), - rot_omega.Diff(x)))

grad_sigma_manufactured = CF((-div_omega.Diff(x), -div_omega.Diff(y)))

f_rhs = CF(curl_rot_omega - grad_div_omega)
f = LinearForm(fes)
f += f_rhs * eta * dx



- Assemble the matrix $A$ and vector $f$
- Calculate the condition number of $A$ just for checking
- Solve the system
- Print relevant stuff!

In [10]:
a.Assemble()
f.Assemble()

rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
cond_nr = np.linalg.cond(A.todense())

sol = GridFunction(fes)
res = f.vec-a.mat * sol.vec
inv = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
sol.vec.data += inv * res

gf_omega , gf_sigma = sol.components

curl_omega = curl(gf_omega)
grad_sigma = grad(gf_sigma)

print("Matrix dimensions:", a.mat.height, "x", a.mat.width)
print("Matrix Condition Number: ", cond_nr)
print("Residual: ", Norm(res))
print("L2 Error omega:", sqrt(Integrate((gf_omega - omega_exact)**2, mesh)))
print("L2 Error curl(omega)", sqrt(Integrate((curl_omega - rot_omega)**2, mesh)))
print("L2 Error sigma:", sqrt(Integrate((gf_sigma + div_omega)**2, mesh)))
print("L2 Error grad(sigma):", sqrt(Integrate((grad_sigma - grad_sigma_manufactured)**2, mesh)))

#print("Condition Number: ", cond_nr)

Matrix dimensions: 493 x 493
Matrix Condition Number:  113168760.70512922
Residual:  7.52652087188852e-14
L2 Error omega: 0.07106492520899141
L2 Error curl(omega) 0.23061796527501546
L2 Error sigma: 0.0471974643257337
L2 Error grad(sigma): 1.779951436029596


In [14]:
error_field = GridFunction(fes)
error_field = gf_omega - omega_exact
Draw(error_field, mesh, "error")
#


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene